In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.preprocessing import MultiLabelBinarizer


In [ ]:
df = pd.read_csv("custom_team - custom_team.csv")
df

,game_date,wl_home,home_team,away_team,home_team_players,away_team_players,official_id,home_obpm_sum,away_obpm_sum,obpm_comparison,...,away_opp_ft_percent_opponent_100,away_opp_orb_per_100_poss,away_opp_drb_per_100_poss,away_opp_trb_per_100_poss,away_opp_ast_per_100_poss,away_opp_stl_per_100_poss,away_opp_blk_per_100_poss,away_opp_tov_per_100_poss,away_opp_pf_per_100_poss,away_opp_pts_per_100_poss
0,2019-10-23,0,Indiana Pacers,Detroit Pistons,"{4096, 4292, 4102, 4582, 4365, 4237, 4400, 449...","{4064, 4451, 4294, 4043, 4272, 4594, 4050, 421...",1830.0,-25.2,-32.1,Home OBPM > Away OBPM,...,0.755,9.3,35.2,44.5,24.1,7.0,5.2,14.3,21.7,109.2
1,2019-11-08,1,Indiana Pacers,Detroit Pistons,"{4292, 4102, 4582, 4365, 4237, 4400, 4566, 4251}","{4064, 4672, 4451, 4043, 4272, 4594, 4212, 431...",1628954.0,-15.9,-35.8,Home OBPM > Away OBPM,...,0.755,9.3,35.2,44.5,24.1,7.0,5.2,14.3,21.7,109.2
2,2019-11-12,1,Miami Heat,Detroit Pistons,"{4611, 3491, 4746, 4177, 4116, 4023, 4472, 4601}","{4064, 4672, 4451, 4294, 4043, 4272, 4594, 421...",101283.0,-18.9,-40.3,Home OBPM > Away OBPM,...,0.755,9.3,35.2,44.5,24.1,7.0,5.2,14.3,21.7,109.2
3,2020-03-11,1,Philadelphia 76ers,Detroit Pistons,"{4417, 3982, 4110, 4336, 4501, 4118, 4055, 476...","{4672, 4451, 4714, 4333, 4272, 3985, 4785, 421...",2148.0,-35.3,-27.5,Home OBPM <= Away OBPM,...,0.755,9.3,35.2,44.5,24.1,7.0,5.2,14.3,21.7,109.2
4,2019-11-15,1,Charlotte Hornets,Detroit Pistons,"{4704, 3624, 4366, 3984, 4659, 4155, 4541, 4607}","{4064, 4672, 4451, 4043, 4272, 3920, 4594, 421...",202901.0,0.6,-35.8,Home OBPM > Away OBPM,...,0.755,9.3,35.2,44.5,24.1,7.0,5.2,14.3,21.7,109.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4163,2022-11-02,1,Cleveland Cavaliers,Boston Celtics,"{4931, 4515, 4708, 4710, 3845, 4479, 4838, 449...","{4518, 5003, 4492, 4720, 4275, 3734, 4415}",1179.0,-19.7,-20.1,Home OBPM > Away OBPM,...,0.783,10.7,34.0,44.7,22.1,7.7,4.7,14.1,19.9,106.9
4164,2023-03-06,1,Cleveland Cavaliers,Boston Celtics,"{4931, 4515, 4708, 4710, 4479, 4838, 4495, 485...","{5003, 4492, 4875, 3920, 4720, 4275, 4183, 4415}",1146.0,-13.4,-8.4,Home OBPM <= Away OBPM,...,0.783,10.7,34.0,44.7,22.1,7.7,4.7,14.1,19.9,106.9
4165,2022-11-05,0,New York Knicks,Boston Celtics,"{4960, 4837, 4871, 4782, 4627, 4630, 4086, 4697}","{4518, 5003, 4492, 3920, 4720, 4275, 4282, 4415}",2534.0,-23.0,-9.9,Home OBPM <= Away OBPM,...,0.783,10.7,34.0,44.7,22.1,7.7,4.7,14.1,19.9,106.9
4166,2023-02-27,1,New York Knicks,Boston Celtics,"{4837, 4871, 5000, 4782, 4627, 4661, 4630, 4991}","{4518, 5003, 4492, 4875, 3920, 4720, 4275, 373...",1155.0,-20.3,-17.8,Home OBPM <= Away OBPM,...,0.783,10.7,34.0,44.7,22.1,7.7,4.7,14.1,19.9,106.9


In [ ]:
df["wl_home"].value_counts()

1    2312
0    1856
Name: wl_home, dtype: int64

In [ ]:
# Calculate percentage of nulls in each column
percent_nulls = (df.isnull().sum() / len(df)) * 100

# Filter to display only columns with null values
columns_with_nulls = percent_nulls[percent_nulls > 0]
print("Percentage of nulls in columns with null values:\n", columns_with_nulls)

Percentage of nulls in columns with null values:
 official_id    9.524952
dtype: float64


In [ ]:
df = df.drop(columns=['official_id'])
df['game_date'] = pd.to_datetime(df['game_date'])

In [ ]:
column_names = df.columns.tolist()
selected_columns = [col for col in column_names if 'lg' in col or 'abbreviation' in col]
df = df.drop(columns=selected_columns)
df.shape

(4168, 253)

In [ ]:
def convert_to_list(set_str):
    return list(eval(set_str))

# Convert the 'home_team_players' and 'away_team_players' from set strings to lists
df['home_team_players'] = df['home_team_players'].apply(convert_to_list)
df['away_team_players'] = df['away_team_players'].apply(convert_to_list)

mlb = MultiLabelBinarizer()

# Fit and transform the home_team_players
home_teams_encoded = mlb.fit_transform(df['home_team_players'])

# Transform the away_team_players
away_teams_encoded = mlb.transform(df['away_team_players'])

# Create DataFrame from encoded data
home_teams_df = pd.DataFrame(home_teams_encoded, columns=mlb.classes_)
away_teams_df = pd.DataFrame(away_teams_encoded, columns=mlb.classes_)

# Prefix columns to distinguish between home and away
home_teams_df = home_teams_df.add_prefix('home_')
away_teams_df = away_teams_df.add_prefix('away_')

# Concatenate the home and away dataframes to the original df
df_encoded = pd.concat([df, home_teams_df, away_teams_df], axis=1)

C:\Program Files\Anaconda\lib\site-packages\sklearn\preprocessing\_label.py:895: UserWarning: unknown class(es) [3258, 3833, 3850, 4201, 4517, 457, 4612, 4666, 4750, 4812, 4830, 4928, 5007, 5028, 5042, 5099] will be ignored
  warnings.warn(


In [ ]:
df_encoded = df_encoded.drop(columns = ['home_team_players', 'away_team_players'] )
df_encoded.head()  # Display the first few rows of the encoded dataframe

,game_date,wl_home,home_team,away_team,home_obpm_sum,away_obpm_sum,obpm_comparison,home_dbpm_sum,away_dbpm_sum,dbpm_comparison,...,away_5095,away_5097,away_5098,away_5100,away_5102,away_5103,away_5104,away_5105,away_5106,away_5107
0,2019-10-23,0,Indiana Pacers,Detroit Pistons,-25.2,-32.1,Home OBPM > Away OBPM,-10.3,-2.4,Home DBPM <= Away DBPM,...,0,0,0,0,0,0,0,0,0,0
1,2019-11-08,1,Indiana Pacers,Detroit Pistons,-15.9,-35.8,Home OBPM > Away OBPM,-5.4,-5.2,Home DBPM <= Away DBPM,...,0,0,0,0,0,0,0,0,0,0
2,2019-11-12,1,Miami Heat,Detroit Pistons,-18.9,-40.3,Home OBPM > Away OBPM,-56.6,-5.2,Home DBPM <= Away DBPM,...,0,0,0,0,0,0,0,0,0,0
3,2020-03-11,1,Philadelphia 76ers,Detroit Pistons,-35.3,-27.5,Home OBPM <= Away OBPM,-7.3,-10.0,Home DBPM > Away DBPM,...,0,0,0,0,0,0,0,0,0,0
4,2019-11-15,1,Charlotte Hornets,Detroit Pistons,0.6,-35.8,Home OBPM > Away OBPM,4.0,-5.7,Home DBPM > Away DBPM,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
data_types = df_encoded.dtypes
categorical_columns = [col for col, dtype in data_types.items() if dtype == 'object' or dtype.name == 'category']

print(categorical_columns)

['home_team', 'away_team', 'obpm_comparison', 'dbpm_comparison']


In [ ]:
df_f = pd.get_dummies(df_encoded, columns=categorical_columns)
df_f

,game_date,wl_home,home_obpm_sum,away_obpm_sum,home_dbpm_sum,away_dbpm_sum,season,home_playoffs_team,home_g_team,home_mp,...,away_team_Portland Trail Blazers,away_team_Sacramento Kings,away_team_San Antonio Spurs,away_team_Toronto Raptors,away_team_Utah Jazz,away_team_Washington Wizards,obpm_comparison_Home OBPM <= Away OBPM,obpm_comparison_Home OBPM > Away OBPM,dbpm_comparison_Home DBPM <= Away DBPM,dbpm_comparison_Home DBPM > Away DBPM
0,2019-10-23,0,-25.2,-32.1,-10.3,-2.4,2019,True,82,19705,...,0,0,0,0,0,0,0,1,1,0
1,2019-11-08,1,-15.9,-35.8,-5.4,-5.2,2019,True,82,19705,...,0,0,0,0,0,0,0,1,1,0
2,2019-11-12,1,-18.9,-40.3,-56.6,-5.2,2019,False,82,19730,...,0,0,0,0,0,0,0,1,1,0
3,2020-03-11,1,-35.3,-27.5,-7.3,-10.0,2019,True,82,19805,...,0,0,0,0,0,0,1,0,0,1
4,2019-11-15,1,0.6,-35.8,4.0,-5.7,2019,False,82,19830,...,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4163,2022-11-02,1,-19.7,-20.1,-10.7,0.5,2022,False,82,19730,...,0,0,0,0,0,0,0,1,1,0
4164,2023-03-06,1,-13.4,-8.4,-10.8,-1.2,2022,False,82,19730,...,0,0,0,0,0,0,1,0,1,0
4165,2022-11-05,0,-23.0,-9.9,-6.8,2.6,2022,False,82,19780,...,0,0,0,0,0,0,1,0,1,0
4166,2023-02-27,1,-20.3,-17.8,-7.4,-2.6,2022,False,82,19780,...,0,0,0,0,0,0,1,0,1,0


In [ ]:
#  Calculate the correlation matrix
corr_matrix = df_f.corr()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(pd.np.triu(pd.np.ones(corr_matrix.shape), k=1).astype(pd.np.bool))

# Find index/column names of highly correlated features (above the threshold)
threshold = 0.80
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

# Print highly correlated columns pairs and their correlations for review
print("Highly correlated columns (above threshold of 0.80):")
for col in to_drop:
    correlated_cols = upper.index[upper[col] > threshold].tolist()
    for row in correlated_cols:
        print(f"{row} and {col} correlation: {upper.at[row, col]}")

# Drop features
df_reduced = df_f.drop(columns=to_drop)

# Shape of the original and reduced dataframes
print("Reduced DataFrame shape:", df_reduced.shape)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_12732\3794205046.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = df_f.corr()
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_12732\3794205046.py:5: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  upper = corr_matrix.where(pd.np.triu(pd.np.ones(corr_matrix.shape), k=1).astype(pd.np.bool))
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_12732\3794205046.py:5: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/

Highly correlated columns (above threshold of 0.80):
home_g_team and home_mp correlation: 0.9989637792400563
home_fg_per_100_poss and home_fg_percent_team correlation: 0.8329404371856591
home_x3p_per_100_poss and home_x3pa_per_100_poss correlation: 0.9345746440267996
home_x2p_per_100_poss and home_x2pa_per_100_poss correlation: 0.8722610346514456
home_ft_per_100_poss and home_fta_per_100_poss correlation: 0.8778680624656567
home_drb_per_100_poss and home_trb_per_100_poss correlation: 0.8181148786741379
home_playoffs_team and home_playoffs_gm correlation: 1.0
home_g_team and home_g_gm correlation: 1.0
home_mp and home_g_gm correlation: 0.998963779240056
home_fg_per_100_poss and home_fg_per_game correlation: 0.8318068391860338
home_fg_per_100_poss and home_fg_percent_gm correlation: 0.8329404371856591
home_fg_percent_team and home_fg_percent_gm correlation: 1.0
home_x3p_per_100_poss and home_x3p_per_game correlation: 0.9847149988678583
home_x3pa_per_100_poss and home_x3p_per_game correla

In [ ]:
if 'game_date' in df_reduced.columns:
    df_reduced['game_date'] = pd.to_datetime(df_reduced['game_date']).map(pd.Timestamp.toordinal)

# Convert boolean columns to int (if any)
bool_cols = df_reduced.select_dtypes(include=['bool']).columns
df_reduced[bool_cols] = df_reduced[bool_cols].astype(int)

In [ ]:
X = df_reduced.drop(columns=['wl_home'])  # Features
y = df_reduced['wl_home']  # Target variable

X.columns = [col.replace('[', '_').replace(']', '_').replace('<', '_') for col in X.columns]
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the shapes of the training and testing sets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


X_train shape: (3334, 1324)
X_test shape: (834, 1324)
y_train shape: (3334,)
y_test shape: (834,)


# Train

In [ ]:
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    "CatBoost": CatBoostClassifier(verbose=0),
    "LightGBM": LGBMClassifier()
}

# Train and evaluate models
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    results[name] = {"Accuracy": accuracy, "ROC AUC": roc_auc, "Classification Report": report}

# Print results for each model
for model_name, metrics in results.items():
    print(f"Results for {model_name}:")
    print(f"Accuracy: {metrics['Accuracy']:.2f}")
    print(f"ROC AUC: {metrics['ROC AUC']:.2f}")
    print(metrics['Classification Report'])
    print("\n---\n")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1838, number of negative: 1496
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.024316 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6750
[LightGBM] [Info] Number of data points in the train set: 3334, number of used features: 864
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.551290 -> initscore=0.205883
[LightGBM] [Info] Start training from score 0.205883
Results for Logistic Regression:
Accuracy: 0.55
ROC AUC: 0.51
              precision    recall  f1-score   support

           0       0.46      0.23      0.31       360
           1       0.58      0.80      0.67       474

    accuracy                           0.55       834
   macro avg       0.52      0.51      0.49       834
weighted avg       0.53      0.55      0.51       834


---

Results for Random Forest:
Accuracy

In [ ]:
model = CatBoostClassifier(iterations=500, depth=3, learning_rate=0.1, loss_function='Logloss', verbose=True)
model.fit(X_train, y_train)

# Get feature importances
feature_importances = model.get_feature_importance()
feature_names = X_train.columns
feature_scores = pd.DataFrame(list(zip(feature_names, feature_importances)), columns=['Feature', 'Importance'])

# Sort features by importance
top_features = feature_scores.sort_values(by='Importance', ascending=False).head(500)

# Select top 50 features
top_features_names = top_features['Feature'].values
X_train_top = X_train[top_features_names]
X_test_top = X_test[top_features_names]

# Retrain model on top features only
model_top = CatBoostClassifier(iterations=500, depth=3, learning_rate=0.1, loss_function='Logloss', verbose=True)
model_top.fit(X_train_top, y_train)

0:	learn: 0.6897814	total: 8.6ms	remaining: 4.29s
1:	learn: 0.6850851	total: 18.6ms	remaining: 4.63s
2:	learn: 0.6809376	total: 29.1ms	remaining: 4.83s
3:	learn: 0.6775312	total: 38.1ms	remaining: 4.72s
4:	learn: 0.6743716	total: 45.6ms	remaining: 4.52s
5:	learn: 0.6717616	total: 52.4ms	remaining: 4.32s
6:	learn: 0.6692130	total: 58.9ms	remaining: 4.14s
7:	learn: 0.6669546	total: 65.6ms	remaining: 4.03s
8:	learn: 0.6648216	total: 71.9ms	remaining: 3.92s
9:	learn: 0.6626708	total: 78.8ms	remaining: 3.86s
10:	learn: 0.6612295	total: 84.7ms	remaining: 3.77s
11:	learn: 0.6598991	total: 90.8ms	remaining: 3.69s
12:	learn: 0.6586746	total: 96.5ms	remaining: 3.62s
13:	learn: 0.6572081	total: 102ms	remaining: 3.53s
14:	learn: 0.6555523	total: 107ms	remaining: 3.45s
15:	learn: 0.6538324	total: 112ms	remaining: 3.38s
16:	learn: 0.6523765	total: 117ms	remaining: 3.31s
17:	learn: 0.6515220	total: 121ms	remaining: 3.25s
18:	learn: 0.6504359	total: 126ms	remaining: 3.2s
19:	learn: 0.6494473	total: 13

196:	learn: 0.5388845	total: 1.02s	remaining: 1.56s
197:	learn: 0.5382320	total: 1.02s	remaining: 1.56s
198:	learn: 0.5377035	total: 1.03s	remaining: 1.55s
199:	learn: 0.5374021	total: 1.03s	remaining: 1.55s
200:	learn: 0.5369283	total: 1.04s	remaining: 1.54s
201:	learn: 0.5366070	total: 1.04s	remaining: 1.54s
202:	learn: 0.5362586	total: 1.05s	remaining: 1.53s
203:	learn: 0.5358497	total: 1.05s	remaining: 1.53s
204:	learn: 0.5352030	total: 1.06s	remaining: 1.52s
205:	learn: 0.5345325	total: 1.06s	remaining: 1.52s
206:	learn: 0.5340564	total: 1.07s	remaining: 1.51s
207:	learn: 0.5332996	total: 1.07s	remaining: 1.51s
208:	learn: 0.5329141	total: 1.08s	remaining: 1.5s
209:	learn: 0.5322844	total: 1.08s	remaining: 1.5s
210:	learn: 0.5317523	total: 1.09s	remaining: 1.49s
211:	learn: 0.5310931	total: 1.09s	remaining: 1.48s
212:	learn: 0.5307840	total: 1.1s	remaining: 1.48s
213:	learn: 0.5302256	total: 1.1s	remaining: 1.47s
214:	learn: 0.5299126	total: 1.11s	remaining: 1.47s
215:	learn: 0.52

359:	learn: 0.4715936	total: 1.83s	remaining: 712ms
360:	learn: 0.4713974	total: 1.83s	remaining: 706ms
361:	learn: 0.4708253	total: 1.84s	remaining: 701ms
362:	learn: 0.4706114	total: 1.84s	remaining: 696ms
363:	learn: 0.4701107	total: 1.85s	remaining: 691ms
364:	learn: 0.4697985	total: 1.85s	remaining: 686ms
365:	learn: 0.4693883	total: 1.86s	remaining: 681ms
366:	learn: 0.4691435	total: 1.86s	remaining: 675ms
367:	learn: 0.4687156	total: 1.87s	remaining: 670ms
368:	learn: 0.4682122	total: 1.87s	remaining: 665ms
369:	learn: 0.4679000	total: 1.88s	remaining: 660ms
370:	learn: 0.4675814	total: 1.88s	remaining: 655ms
371:	learn: 0.4673187	total: 1.89s	remaining: 650ms
372:	learn: 0.4669651	total: 1.89s	remaining: 645ms
373:	learn: 0.4667215	total: 1.9s	remaining: 640ms
374:	learn: 0.4666525	total: 1.9s	remaining: 634ms
375:	learn: 0.4662383	total: 1.91s	remaining: 629ms
376:	learn: 0.4657402	total: 1.91s	remaining: 624ms
377:	learn: 0.4653469	total: 1.92s	remaining: 619ms
378:	learn: 0.

47:	learn: 0.6257430	total: 209ms	remaining: 1.96s
48:	learn: 0.6249192	total: 213ms	remaining: 1.96s
49:	learn: 0.6237540	total: 217ms	remaining: 1.95s
50:	learn: 0.6233388	total: 221ms	remaining: 1.94s
51:	learn: 0.6227239	total: 224ms	remaining: 1.93s
52:	learn: 0.6223747	total: 228ms	remaining: 1.92s
53:	learn: 0.6218050	total: 231ms	remaining: 1.91s
54:	learn: 0.6213757	total: 235ms	remaining: 1.9s
55:	learn: 0.6209139	total: 240ms	remaining: 1.9s
56:	learn: 0.6202653	total: 243ms	remaining: 1.89s
57:	learn: 0.6197706	total: 247ms	remaining: 1.88s
58:	learn: 0.6191998	total: 251ms	remaining: 1.88s
59:	learn: 0.6187084	total: 255ms	remaining: 1.87s
60:	learn: 0.6180440	total: 259ms	remaining: 1.86s
61:	learn: 0.6175717	total: 263ms	remaining: 1.86s
62:	learn: 0.6170735	total: 267ms	remaining: 1.85s
63:	learn: 0.6165330	total: 271ms	remaining: 1.84s
64:	learn: 0.6158510	total: 275ms	remaining: 1.84s
65:	learn: 0.6155189	total: 278ms	remaining: 1.82s
66:	learn: 0.6148107	total: 282ms

207:	learn: 0.5336820	total: 819ms	remaining: 1.15s
208:	learn: 0.5331227	total: 823ms	remaining: 1.15s
209:	learn: 0.5327834	total: 827ms	remaining: 1.14s
210:	learn: 0.5322688	total: 830ms	remaining: 1.14s
211:	learn: 0.5316557	total: 834ms	remaining: 1.13s
212:	learn: 0.5311303	total: 838ms	remaining: 1.13s
213:	learn: 0.5304416	total: 842ms	remaining: 1.13s
214:	learn: 0.5301236	total: 846ms	remaining: 1.12s
215:	learn: 0.5296055	total: 850ms	remaining: 1.12s
216:	learn: 0.5290480	total: 854ms	remaining: 1.11s
217:	learn: 0.5284160	total: 858ms	remaining: 1.11s
218:	learn: 0.5282347	total: 861ms	remaining: 1.1s
219:	learn: 0.5277133	total: 866ms	remaining: 1.1s
220:	learn: 0.5271201	total: 870ms	remaining: 1.1s
221:	learn: 0.5261252	total: 873ms	remaining: 1.09s
222:	learn: 0.5256989	total: 877ms	remaining: 1.09s
223:	learn: 0.5251281	total: 880ms	remaining: 1.08s
224:	learn: 0.5247120	total: 884ms	remaining: 1.08s
225:	learn: 0.5243072	total: 888ms	remaining: 1.08s
226:	learn: 0.5

365:	learn: 0.4696075	total: 1.42s	remaining: 519ms
366:	learn: 0.4693635	total: 1.42s	remaining: 515ms
367:	learn: 0.4690266	total: 1.43s	remaining: 511ms
368:	learn: 0.4684932	total: 1.43s	remaining: 508ms
369:	learn: 0.4681406	total: 1.43s	remaining: 504ms
370:	learn: 0.4675782	total: 1.44s	remaining: 500ms
371:	learn: 0.4672534	total: 1.44s	remaining: 496ms
372:	learn: 0.4671787	total: 1.44s	remaining: 492ms
373:	learn: 0.4667687	total: 1.45s	remaining: 488ms
374:	learn: 0.4665986	total: 1.45s	remaining: 484ms
375:	learn: 0.4662982	total: 1.46s	remaining: 480ms
376:	learn: 0.4658700	total: 1.46s	remaining: 476ms
377:	learn: 0.4654358	total: 1.46s	remaining: 472ms
378:	learn: 0.4651617	total: 1.47s	remaining: 469ms
379:	learn: 0.4646712	total: 1.47s	remaining: 465ms
380:	learn: 0.4643464	total: 1.48s	remaining: 461ms
381:	learn: 0.4640146	total: 1.48s	remaining: 457ms
382:	learn: 0.4636643	total: 1.48s	remaining: 453ms
383:	learn: 0.4632975	total: 1.49s	remaining: 450ms
384:	learn: 

In [ ]:
predictions = model_top.predict(X_test_top)
print("Accuracy:", accuracy_score(y_test, predictions))

Accuracy: 0.6199040767386091
